In [1]:
from logic import Literal, Clause, Rule

In [2]:
happy, nothappy = Literal("happy", True), Literal("happy", False)  # Literals print in prolog syntax
print("We can create literals {} and {}".format( happy, nothappy ),
      "that consist of the same atom 'happy', but opposite signs.")

We can create literals happy and ~happy that consist of the same atom 'happy', but opposite signs.


In [3]:
sunny, stay_home = Literal("sunny"), Literal("stay_home")  # We can omit signs for positive literals.
c1 = Clause(sunny, stay_home) # Clauses print in prolog syntax
print("Clauses are made up of literals;", c1) 

Clauses are made up of literals; stay_home, sunny.


In [4]:
r1 = Rule(nothappy, sunny, stay_home)  # The first literal is the head, the body literals follow.
print("Rules have a head literal, and body literals;", r1)  # Rules print in prolog sytax

Rules have a head literal, and body literals; ~happy:- stay_home, sunny.


In [5]:
from knowledgebase import KnowledgeBase, PrologString

In [6]:
print("We can create knowledge bases too;\n", KnowledgeBase(r1, c1))

We can create knowledge bases too;
 stay_home, sunny.
~happy:- stay_home, sunny.



In [7]:
ps = PrologString("~happy:- stay_home, sunny."  # PrologStrings don't print in prolog syntax
                + "stay_home, sunny. ")
print("Knowledge bases can also be represented as PrologString objects;",
      *ps.clauses.union(ps.rules), sep="\n")
del ps  #cleanup

Knowledge bases can also be represented as PrologString objects;
stay_home, sunny.
~happy:- stay_home, sunny.


In [8]:
from argumentation import Case

In [9]:
notwork_well = Literal("work_well", False)
r2 = Rule(notwork_well, stay_home)
kb = KnowledgeBase(r2, c1)
print("For a knowledge base containing:\n", kb)

l = kb._literals_dict["~work_well"]  # Yields kb's instance of notwork_well as l
r = list(l.case.asserting_rules)[0]  # Yields the (only) rule in kb that asserts l, as r
b = list(r.body)[0]  # Yields r's (only) body literal
c = list(b.case.asserting_clauses)[0]  # Yields the (only) clause asserting b, as c
print("The literal {} is asserted by the rule {}".format(l, r),
      "\nThis rule is supported by the clause {}".format(c))

print("\nNote that this is everything you need to get the argument:\n([{}, {}], {})".format(r, c, l))
del l, r, b, c  # cleanup

For a knowledge base containing:
 stay_home, sunny.
~work_well:- stay_home.

The literal ~work_well is asserted by the rule ~work_well:- stay_home. 
This rule is supported by the clause stay_home, sunny.

Note that this is everything you need to get the argument:
([~work_well:- stay_home., stay_home, sunny.], ~work_well)


In [10]:
r3, r4 = Rule(happy, stay_home), Rule(Literal("work_well"), happy)  # need to create a couple more rules
print("We can populate a KnowledgeBase with the Clause and Rule objects we've made so far (& then some):")
kb = KnowledgeBase(c1, r1, r2, r3, r4)
print("KB:", str(kb).replace("\n", "\n    "))

We can populate a KnowledgeBase with the Clause and Rule objects we've made so far (& then some):
KB: stay_home, sunny.
    happy:- stay_home.
    ~work_well:- stay_home.
    work_well:- happy.
    ~happy:- stay_home, sunny.
    


In [11]:
print("Our notwork_well =", notwork_well)
w = kb._literals_dict["~work_well"]  # get kb's literal ~work_well
print("KB's  ~work_well =", w)
print("Shared instance?:", w is notwork_well)

Our notwork_well = ~work_well
KB's  ~work_well = ~work_well
Shared instance?: False


In [12]:
print("Our  r2:", r2)
r = list(w.case.asserting_rules)[0]  # get kb's r2; the only rule that asserts ~work_well
print("KB's r2:", r)
print("Shared instance?:", r is r2)

Our  r2: ~work_well:- stay_home.
KB's r2: ~work_well:- stay_home.
Shared instance?: False


In [13]:
s1 = list(r.body)[0] # get kb's r2's only body literal (stay_home)
print("KB's r2's body literal:", s1)
h = kb._literals_dict["happy"]  # get the literal happy
r = list(h.case.asserting_rules)[0]  # get kb's r3; the only rule that asserts happy
s2 = list(r.body)[0] # get kb's r3's only body literal (stay_home)
print("KB's r3's body literal:", s2)
print("Shared instance?:", s1 is s2)
del w, r, s1, h, s2

KB's r2's body literal: stay_home
KB's r3's body literal: stay_home
Shared instance?: True


In [14]:
s = "~happy:- sunny, stay_home. ~work_well:- stay_home. happy:- stay_home. work_well:- happy."
s += "stay_home, sunny."
print("We can populate a knowledge base via a prolog-syntax string.",
      "E.G. for the same contents:",
       KnowledgeBase(s), sep="\n")

We can populate a knowledge base via a prolog-syntax string.
E.G. for the same contents:
stay_home, sunny.
happy:- stay_home.
~work_well:- stay_home.
work_well:- happy.
~happy:- stay_home, sunny.



In [15]:
with open("kb_input_file.txt", "w") as infile:
    infile.write(s)
print("We can populate a knowledge base via a text file containing a prolog-syntax string.",
      "E.G. for the same contents:",
       KnowledgeBase("kb_input_file.txt"), sep="\n")

We can populate a knowledge base via a text file containing a prolog-syntax string.
E.G. for the same contents:
stay_home, sunny.
happy:- stay_home.
~work_well:- stay_home.
work_well:- happy.
~happy:- stay_home, sunny.



In [16]:
from itertools import product
from functools import reduce
def rule_supporting_evidence(r):
    """
    Returns a collection (generator) of sets of supporting evidence for Rule r.

    Equivalent to:
    antecedent_evidence_items = [literal_supporting_evidence(l.case) for l in r.body]
    for antecedent_evidence_item in product(*antecedent_evidence_items):
        yield reduce(set().union, antecedent_evidence_item).union({r})
    """
    yield from (reduce(set().union, ccose).union({r})  for ccose in product(*(literal_supporting_evidence(l) for l in r.body)))

In [17]:
from itertools import chain
def literal_supporting_evidence(l):
    """
    Returns a collection (generator) of sets of supporting evidence for the Literal l.
    """
    c = l.case
    # base step: yield each asserting clause of l as a set
    yield from ({clause} for clause in c.asserting_clauses)
    # recursive step: yield each collection of supporting evidence for
    #     (supported) asserted rules of l (as a set)
    yield from chain.from_iterable((rule_supporting_evidence(r) for r in c.supporting_rules))

In [18]:
for l in kb._supported_literals.values():
    print("\nArguments for {}:".format(l))
    supporting_evidence = chain.from_iterable((rule_supporting_evidence(r) for r in l.case.supporting_rules))
    print(*supporting_evidence, sep=" ")


Arguments for ~happy:
{stay_home, sunny., ~happy:- stay_home, sunny.}

Arguments for ~work_well:
{stay_home, sunny., ~work_well:- stay_home.}

Arguments for happy:
{stay_home, sunny., happy:- stay_home.}

Arguments for work_well:
{stay_home, sunny., happy:- stay_home., work_well:- happy.}
